## Проект по НИСу "Не мой язык". Настя Макунина, Таня Мамонова, Женя Рубанова

### Нужно было повторить эксперимент, описанный в статье Dickinson, Herring (2008) -- использовать FSA для локализации ошибки в слове 

## Чистим данные

In [1]:
import csv
import re

In [2]:
with open('morphdict_verb.csv', 'r', encoding = 'utf-8') as c:
    bad_dict = csv.reader(c, delimiter=';')
    bad_dict = list(bad_dict)

In [3]:
bad_dict[0]

['Lemma',
 'Morph',
 'Status',
 'Plaсе',
 'Allomorph',
 'POS',
 'CommentWord',
 'CommentMorph']

In [4]:
d = []
d = bad_dict[1:]

"""
bad_dict = bad_dict[1:]
verb_marks = ['ь', 'ся', 'ти']
for row in bad_dict:
    for mark in verb_marks:
        if row[0].endswith(mark):
            d.append(row)
"""

"\nbad_dict = bad_dict[1:]\nverb_marks = ['ь', 'ся', 'ти']\nfor row in bad_dict:\n    for mark in verb_marks:\n        if row[0].endswith(mark):\n            d.append(row)\n"

In [5]:
d = [x[:5] for x in d]

In [6]:
d = [[x[0], x[1], x[2], x[3], re.sub('[0-9]', '', x[4])] for x in d]
d = [[x[0], x[1], x[2], x[3], re.sub('\(.*\)', '', x[4])] for x in d]

### Сначала мы решили сделать FSA, но он не считал ошибками случаи типа "покуреть", т.е. когда суффикс существует, но в конкретном слове он неверен

### Поэтому мы сделали граф, который будет определять, может ли та или иная морфема быть в слове. На вход подается словарь, мы сравниваем в каждой строке нулевой элемент словаря - слово, если он совпадает с последующим, то мы добавляем первый элемент, который содержит в себе морфему в граф. 

In [7]:
import networkx as nx

In [8]:
def make_graph(d, restricted_morphs={"0", "", None}):
    G = nx.Graph()
    
    def add_morph_edge(morph1, morph2):
        if all((morph1, morph2 not in restricted_morphs)):
            G.add_edge(morph1, morph2)
    
    start_word = "<START>"
    end_word = "<END>"
    prev_word = None
    prev_morph = None
    d.append([None, None])
    
    for el in d:
        word = el[0]
        morph = el[1]
        if prev_word != word:
            add_morph_edge(prev_morph, end_word)
            add_morph_edge(start_word, morph)
            prev_word = word
        else:
            add_morph_edge(prev_morph, morph)
        prev_morph = morph
    
    d.pop()
    return G

In [9]:
%%time

G = make_graph(d)

Wall time: 584 ms


In [10]:
print(G.has_edge("кур", "и"))
print(G.has_edge("кур", "е"))

True
False


## делаем forward - обычный FSA

### Сначала сделаем списки морфем, которые встретились в нашем словаре и добавим аффиксы, которых там нет

In [11]:
roots = []
suffs = []
prefs = []
flex = []

for row in d:
    if row[2] == 'суффикс':
        suffs.append(row[1])
    elif row[2] == 'префикс':
        prefs.append(row[1])
    elif row[2] == 'флексия':
        flex.append(row[1])
    elif row[2] == 'корень':
        if row[4] != '':
            arr = row[4].split('|')
            arr = [re.sub('[\[\]]', '', x) for x in arr]
            for i in arr:
                roots.append(i)
        else:
            roots.append(row[1])

In [12]:
prefs = set(prefs)
roots = set(roots)
suffs = set(suffs)
flex = set(flex)

In [13]:
print('Кол-во префиксов: ', len(prefs))
print('Кол-во суффиксов: ', len(suffs))
print('Кол-во окончаний: ', len(flex))
print('Кол-во корней: ', len(roots))

Кол-во префиксов:  90
Кол-во суффиксов:  171
Кол-во окончаний:  7
Кол-во корней:  3180


In [14]:
flex.remove('ь')
suffs.remove('ь')

In [15]:
flex_to_add = ["ем", "ешь", "ете", "ет", "ут", "ют", "у", "ю", "им", "ишь", "ите", "ит", "ат", "ят", "а", "и", "о", "сь"]

for f in flex_to_add:
    flex.add(f)

In [16]:
suffs_to_add = ["ова", "ева", "ива", 'ирова', "ыва", "а", "и", "я", "е", "л"]

In [17]:
suffs = ['а', 'я', 'к', 'а', 'е', 'ев', 'а', 'ов', 'а', 'и', 'нич', 'а', 'ну', 'ств', 'ов', 'а'] + suffs_to_add
suffs = set(suffs)

In [18]:
prefs = {'взо', 'во', 'в', 'до', 'воз', 'вз', 'вс', 'вы', 'возо', 'за', 'изо', 'ко', 'над', 'надо', 'на', 'недо', 'о', 'об', 'обо', 'от', 'пере', 'по', 'под', 'пред', 'про', 'с', 'со', 'среди', 'через', 'черес', 'раз' ,'рас', 'роз', 'рос', 'без', 'бес', 'оз', 'вос', 'из', 'ис', 'низ', 'нис', 'пре', 'при', 'от', 'ото'}

In [19]:
roots.remove('')

In [20]:
roots.add('блок')

In [21]:
'' in flex

False

### Строим автомат

In [22]:
class StateMachine:
    def __init__(self):
        self.handlers = {}
        self.startState = None
        self.endStates = []

    def add_state(self, name, handler, end_state=0):
        name = name.upper()
        self.handlers[name] = handler
        if end_state:
            self.endStates.append(name)

    def set_start(self, name):
        self.startState = name.upper()

    def run(self, cargo):
        try:
            handler = self.handlers[self.startState]
        except:
            raise InitializationError("must call .set_start() before .run()")
        if not self.endStates:
            raise  InitializationError("at least one state must be an end_state")
    
        while True:
            (newState, cargo) = handler(cargo)
            if newState.upper() in self.endStates:
                print("reached ", newState)
                break 
            else:
                handler = self.handlers[newState.upper()]

In [23]:
#suffs
#prefs
#roots
#flex

analyze = ''
smth = False

## На вход подаем слово, начинаем сопоставлять с префиксами из нашего списка, проверяем на графе, оставшееся записываем в rest

def pref_transitions(word):
    global G
    found = []
    rest = []
    for i in prefs:
        if re.match(i, word):
            found.append(re.match(i, word).group(0))
    if len(found) > 0:
        for f in found:
            if G.has_edge("<START>", f):
                #приставка, остаток
                rest.append([f, word[len(f):]])
    if len(rest) == 0:
        rest = [word]
    newState = 'Pref_state'
    return (newState, rest)

## На вход подаем оставшееся после префикса, начинаем сопоставлять с корнями и с графом, оставшееся записываем в rest

def root_transitions(rest):
    found = []
    #print(rest)
    for item in rest:
        for root in roots:
            if isinstance(item, list): 
                if re.match(root, item[1]):
                    if G.has_edge(item[0], root):
                        #приставка, корень, остаток
                        found.append([item[0], re.match(root, item[1]).group(0), item[1][len(root):]])
            else:
                #print(item)
                if re.match(root, item):
                    if G.has_edge("<START>", root):
                        #корень, остаток
                        found.append([re.match(root, item).group(0), item[len(root):]])
    if len(found) > 0:
        newState = 'Root_state'
        new_rest = []
        for f in found:
            new_rest.append(f)
        rest = new_rest
    else:
        newState = 'PrefRoot_Error'
    return (newState, rest)


def suff_transitions(rest):
    stop = False
    while stop == False:
        rest, stop = find_morphs(rest, suffs)
    newState = 'Suff_state'
    return (newState, rest)
        

## Смотрим, что у нас осталось???????    
    
def find_morphs(rest, morphs):
    global smth
    found = []
    stop = False
    for item in rest:
        for s in morphs:
            if re.match(s, item[-1]):
                if G.has_edge(item[-2], s):
                    #print('в сказку попал')
                    #print(s)
                    #(приставка), корень, суффикс, остаток
                    arr = [item[:-1], [re.match(s, item[-1]).group(0)], [item[-1][len(s):]]]
                    found.append([item for sublist in arr for item in sublist])
                    smth = True
    if found == []:
        stop = True
        found = rest
    #print(found)
    
    return found, stop


def flex_transitions(rest):
    stop = False
    #print('ОКОНЧАНИЯ')
    while stop == False:
        rest, stop = find_morphs(rest, flex)
    #print(rest)
    global analyze
    analyze = rest
    if analyze[0][-1] != '':
        newState = 'SufFlex_Error'
    else:
        newState = 'Flex_state'
    return (newState, rest)
    

if __name__== "__main__":
    m_f = StateMachine()
    m_f.add_state("Start", pref_transitions)
    m_f.add_state("Pref_state", root_transitions)
    m_f.add_state("Root_state", suff_transitions)
    m_f.add_state("Suff_state", flex_transitions)
    m_f.add_state("Flex_state", None, end_state=1)
    m_f.add_state("PrefRoot_Error", None, end_state=1)
    m_f.add_state("SufFlex_Error", None, end_state=1) 
    m_f.set_start("Start")

### Вот как он работает: на вход поступает слово, если оно написано правильно, то автомат достигает Flex_State, если нет, то состояния ошибки с указанием морфемы, в которой ошибка допущена

In [24]:
m_f.run("побежать".lower())
analyze

reached  Flex_state


[['по', 'беж', 'а', 'ть', '']]

In [25]:
m_f.run("побежатб".lower())
analyze

reached  SufFlex_Error


[['по', 'беж', 'а', 'тб']]

In [26]:
m_f.run("побежить".lower())
analyze

reached  SufFlex_Error


[['по', 'беж', 'ить']]

In [27]:
m_f.run("онести".lower())
analyze

reached  SufFlex_Error


[['о', 'н', 'е', 'сти']]

## теперь back - обратный FSA. Он построен по аналогии с прямым автоматом, только анализ слова теперь будет начинаться с окончания

In [28]:
back_flex = set([x[::-1] for x in flex])
back_roots = set([x[::-1] for x in roots])
back_suffs = set([x[::-1] for x in suffs])
back_prefs = set([x[::-1] for x in prefs])

In [29]:
#suffs
#prefs
#roots
#flex

analyze = ''

def back_flex_transitions(rest):
    stop = False
    rest = rest[::-1]
    found = []
    for f in back_flex:
        if re.match(f, rest):
            if G.has_edge(f[::-1], "<END>"):
                found.append([f, rest[len(f):]])
    
    rest, stop = back_find_morphs(found, back_flex)
            
    newState = 'Flex_state'
    return (newState, rest)

def back_find_morphs(rest, morphs):
    #print(rest)
    found = []
    stop = False
    for item in rest:
        for s in morphs:
            if re.match(s, item[-1]):
                if G.has_edge(item[-2][::-1], s[::-1]):
                    #print('в сказку попал')
                    #print(s)
                    #(приставка), корень, суффикс, остаток
                    arr = [item[:-1], [re.match(s, item[-1]).group(0)], [item[-1][len(s):]]]
                    found.append([item for sublist in arr for item in sublist])
                    #print(found)
    if found == []:
        stop = True
        found = rest
    return found, stop

def back_suff_transitions(rest):
    stop = False
    while stop == False:
        rest, stop = find_morphs(rest, back_suffs)
    newState = 'Suff_state'
    return (newState, rest)

def back_root_transitions(rest):
    global analyze
    found = []
    for item in rest:
        for root in back_roots:
            if isinstance(item, list): 
                if re.match(root, item[-1]):
                    if G.has_edge(item[-2][::-1], root[::-1]):
                        arr = [item[:-1], [re.match(root, item[-1]).group(0)], [item[-1][len(root):]]]
                        found.append([item for sublist in arr for item in sublist])
            else:
                #print(item)
                if re.match(root, item):
                    if G.has_edge("<START>", root[::-1]):
                        #корень, остаток
                        found.append([re.match(root, item).group(0), item[len(root):]])
    if len(found) > 0:
        newState = 'Root_state'
        new_rest = []
        for f in found:
            new_rest.append(f)
        rest = new_rest
    else:
        analyze = [[el[::-1] for el in reversed(var)] for var in rest]
        newState = 'SufFlex_Error'
    return (newState, rest)



def back_pref_transitions(rest):
    global analyze
    found = []
    for item in rest:
        for p in back_prefs:
            if re.match(p, item[-1]):
                if G.has_edge(item[-2][::-1], p[::-1]):
                    arr = [item[:-1], [re.match(p, item[-1]).group(0)], [item[-1][len(p):]]]
                    found.append([item for sublist in arr for item in sublist])
    if not found:
        newState = 'PrefRoot_Error'
    else:
        newState = 'Pref_state'
    analyze = [[el[::-1] for el in reversed(var)] for var in rest]
    return (newState, analyze)




    

if __name__== "__main__":
    m_b = StateMachine()
    m_b.add_state("Start", back_flex_transitions)
    m_b.add_state("Flex_state", back_suff_transitions)
    m_b.add_state("Suff_state", back_root_transitions)
    m_b.add_state("Root_state", back_pref_transitions)
    m_b.add_state("Pref_state", None, end_state=1)
    m_b.add_state("PrefRoot_Error", None, end_state=1)
    m_b.add_state("SufFlex_Error", None, end_state=1) 
    m_b.set_start("Start")

### Вот как он работает

In [30]:
m_b.run("побежать".lower())
analyze

reached  Pref_state


[['побе', 'ж', 'а', 'ть'], ['по', 'беж', 'а', 'ть']]

In [31]:
m_b.run("побежатб".lower())
analyze

reached  SufFlex_Error


[]

In [32]:
m_b.run("очерти".lower())
analyze

reached  Pref_state


[['очер', 'т', 'и'], ['о', 'черт', 'и']]

In [33]:
m_b.run("зчерти".lower())
analyze

reached  PrefRoot_Error


[['зчер', 'т', 'и'], ['з', 'черт', 'и']]

In [34]:
m_b.run("онести".lower())
analyze

reached  PrefRoot_Error


[['о', 'нес', 'ти'], ['онес', 'т', 'и']]

In [35]:
m_b.run("побежить".lower())
analyze

reached  PrefRoot_Error


[['побе', 'ж', 'и', 'ть'], ['поб', 'еж', 'и', 'ть']]

### Сравниваем выдачи

In [36]:
m_f.run("побежить".lower())
m_b.run("побежить".lower())

reached  SufFlex_Error
reached  PrefRoot_Error


In [37]:
m_f.run("побежать".lower())
m_b.run("побежать".lower())

reached  Flex_state
reached  Pref_state


Мы видим, что в одном и том же слове в зависимости от того, каким именно автоматом его анализировать, будет разная локализация ошибок. Например, если в слове есть правильная (которая есть в словаре), но не подходящая к корню приставка, то эту ошибку будет детектить только обратный автомат. Соответстенно, наоборот, если то же самое сделать с окончанием, то будет детектить прямой автомат.